<a href="https://colab.research.google.com/github/LuisPerdomo123/Inteligencia-Computacional/blob/main/Classiification_MLP_Caso_Practico_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptrón Multicapa: Caso práctico III

## Dataset
te conjunto de datos contine los dicursos de estos líderes prominentes: Benjamín Netányahu, Jens Stolterberg, Julia Gillard, Margaret Tacher y Nelson Mandela que también representa los nombres de las carpetas. Cada audio de la carpeta es un PCM de un segundo de duración con una velicidad de muestreo de 16000 Hz codificado. Una carpeta llamada background_noise contiene los audios que no son discursos, pero que se pueden encontrar en el interior alrededor del enterno del orador, por ejemplo, la audiencia riendo o aplaudiendo. Se puede mezclar con el discurso mientras se entrena.

### Referencias

https://www.kaggle.com/kongaevans/speaker-recognition-dataset/

## Enunciado
El objetivo del ejercicio consiste en utililzar un perceptrón multicapa para la identificación de la persona que se encuentra hablando a partir de un audio de un segundo de duración.

Si tienes un dataset comprimido en formato .zip que has cargado en Google Colab, puedes extraerlo y luego acceder a los archivos dentro del dataset. Aquí tienes los pasos para hacerlo:

Sube el archivo .zip a Colab: Puedes subir el archivo .zip a Colab utilizando la interfaz de usuario o mediante código. Esto mostrará un botón de "Seleccionar archivos" que te permitirá cargar "16000_pcm_speeches.zip" desde tu sistema local.

Por ejemplo, si el archivo se llama "16000_pcm_speeches.zip" y está en tu sistema local, puedes subirlo así:

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 16000_pcm_speeches.zip to 16000_pcm_speeches.zip


Extrae el archivo .zip: Una vez que hayas subido el archivo .zip, este código extraerá el contenido del archivo "dataset.zip" en el directorio "/content/dataset/" de Colab. Puedes cambiar la ruta de extracción según sea necesario.

puedes extraerlo usando el siguiente código:

In [3]:
import zipfile

with zipfile.ZipFile('16000_pcm_speeches.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

Accede a los archivos del dataset: Ahora que el dataset está descomprimido en el entorno de Colab, puedes acceder a sus archivos normalmente utilizando la ruta donde se extrajo el dataset.

Por ejemplo, si dentro del dataset hay un archivo llamado "archivo.txt", puedes acceder a él así:

In [12]:
# Definimos un conjunto de constantes con las rutas de las carpetas que contienen los audios de cada persona
import os

DATASET_ROOT = "/content/dataset/16000_pcm_speeches"
BENJAMIN_DATA = os.path.join(DATASET_ROOT, "Benjamin_Netanyau")
JENS_DATA = os.path.join(DATASET_ROOT, "Jens_Stoltenberg")
JULIA_DATA = os.path.join(DATASET_ROOT, "Julia_Gillard")
MARGARET_DATA = os.path.join(DATASET_ROOT, "Margaret_Tarcher")
NELSON_DATA = os.path.join(DATASET_ROOT, "Nelson_Mandela")

## 1. Preparación del conjunto de datos

In [15]:
# Definimos una función para parsear nuestro conjunto de datos
import librosa
import numpy as np

def parse_dataset(dataset_paths):
  X = []
  y = []
  for index, dataset in enumerate(dataset_paths):
    print("[+] Parsing {} data...".format(dataset))
    for fname in os.listdir(dataset):
      wav, sr = librosa.load(os.path.join(dataset, fname), sr=None)
      D = librosa.amplitude_to_db(np.abs(librosa.stft(wav)), ref=np.max)
      X.append(D)
      y.append(index)
  return (X, y)

In [17]:
X, y = parse_dataset([BENJAMIN_DATA, JENS_DATA, JULIA_DATA, NELSON_DATA])

[+] Parsing /content/dataset/16000_pcm_speeches/Benjamin_Netanyau data...
[+] Parsing /content/dataset/16000_pcm_speeches/Jens_Stoltenberg data...
[+] Parsing /content/dataset/16000_pcm_speeches/Julia_Gillard data...
[+] Parsing /content/dataset/16000_pcm_speeches/Nelson_Mandela data...


## 2. División del conjunto de datos

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [20]:
X_test, X_val, y_tets, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [21]:
print("Longitud subconjunto de entrenamiento: ", len(X_train))
print("Longitud subconjunto de validación: ", len(X_val))
print("Longitud subconjunto de pruebas: ", len(X_test))

Longitud subconjunto de entrenamiento:  5400
Longitud subconjunto de validación:  301
Longitud subconjunto de pruebas:  300


## 3. Construcción del modelo

In [22]:
from tensorflow.keras.utils import to_categorical

def prep_dataset(X, y, shape):
  X_prep = np.array(X).reshape(len(X), shape)
  X_prep = X_prep.astype('float32') / 255
  y_prep = to_categorical(np.array(y))
  return (X_prep, y_prep)

In [23]:
X_train_prep, y_train_prep = prep_dataset(X_train, y_train, 1025*32)
X_val_prep, y_val_prep = prep_dataset(X_val, y_val, 1025*32)
X_test_prep, y_test_prep = prep_dataset(X_test, y_test, 1025*32)

In [24]:
X_train_prep

array([[-0.2856507 , -0.1980031 , -0.17526971, ..., -0.3137255 ,
        -0.29284686, -0.25467676],
       [-0.14373729, -0.1709079 , -0.22460534, ..., -0.2689801 ,
        -0.22339493, -0.19149806],
       [-0.17463434, -0.13898051, -0.13941142, ..., -0.3137255 ,
        -0.2938944 , -0.26547614],
       ...,
       [-0.28978968, -0.29095238, -0.3137255 , ..., -0.3137255 ,
        -0.29578316, -0.2560798 ],
       [-0.21726583, -0.20812435, -0.19313487, ..., -0.3137255 ,
        -0.2805897 , -0.2444185 ],
       [-0.1539024 , -0.18409081, -0.20397244, ..., -0.3137255 ,
        -0.3137255 , -0.28312832]], dtype=float32)

In [25]:
y_train_prep

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

### 3.1. Entrenamiento de una RNA sencilla

In [26]:
# Entrenamos un perceptrón multicapa
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='logistic', hidden_layer_sizes=(10,), solver='sgd')
clf.fit(X_train_prep, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(10,), solver='sgd')

In [27]:
# Realizamos la predicción con el conjunto de datos de validación
y_pred = clf.predict(X_val_prep)

In [28]:
# Mostramos el accuracy resultante de la clasificación
from sklearn.metrics import accuracy_score

accuracy_score(y_val,y_pred)

0.9534883720930233

### 3.2. Entrenamiento de una RNA profunda

In [30]:
from tensorflow.keras import models
from tensorflow.keras import layers

### 3. Entrenamiento de una RNA profunda

In [31]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [33]:
network = models.Sequential()

network.add(layers.Dense(300, activation='relu', input_shape=(1025*32,)))
network.add(layers.Dense(200, activation='relu'))
network.add(layers.Dense(100, activation='relu'))
network.add(layers.Dense(5, activation='softmax'))

In [34]:
network.compile(loss='categorical_crossentropy',
                optimizer='sgd',
                metrics=['accuracy','Precision'])